# CORRECTEUR D'ORTHOGRAPHE 2


|<h2>Chargement des données textuelles à partir du fichier big.txt</h2> |
| :-------------------------------------------------------------------------: |



In [39]:
from collections import defaultdict
import re
from math import log, exp


#Charger les données
def load_data(nom_fichier: str) -> str:
  
    donnees = []
    with open(nom_fichier, "r", encoding='utf-8') as f:
        donnees = f.read()

    return donnees


|<h2>Tokenisation des données textuelles</h2> |
| :------------------: |
| La fonction process_data effectue une tokenisation des données. Elle commence par diviser les données en phrases, puis elle tokenise chaque phrase individuellement à l'aide de la fonction tokenizer_phrase. Le résultat est un tuple contenant la liste des phrases originales et une liste de listes, où chaque sous-liste représente les tokens d'une phrase spécifique.|


In [40]:
def process_data(donnees: str) -> tuple[list[str], list[list[str]]]:
    """
    Tokenise les données en phrases et en mots.

    Paramètres :
        donnees (str) : Les données d'entrée sous forme de chaîne de caractères.

    Retourne :
        tuple[list[str], list[list[str]]] : Un tuple contenant la liste des phrases et la liste des phrases tokenisées.
    """
    def tokenizer_phrase(phrase: str) -> list[str]:
    
        phrase = phrase.lower()
        phrase = re.sub(r"[^\w\s]", "", phrase)
        tokens = phrase.split()
        return tokens

    #diviser les données en phrases
    phrases = donnees.split('\n')
    phrases = [phrase.strip() for phrase in phrases if phrase.strip()]
    
    #diviser chaque phrase en tokens
    phrases_tokenisees = [tokenizer_phrase(p) for p in phrases]
    
    return phrases, phrases_tokenisees


|<h2>Extraction du vocabulaire à partir de données tokenisées</h2> |
| :------------------: |
| La fonction get_vocabulary prend en entrée une liste de listes de tokens (représentant des phrases tokenisées) et retourne un ensemble contenant tous les mots uniques présents dans ces données. Elle parcourt chaque sous-liste (correspondant à une phrase tokenisée) et ajoute chaque token unique à l'ensemble vocabulaire. Ainsi, à la fin, l'ensemble vocabulaire contient tous les mots uniques utilisés dans les données tokenisées.|


In [41]:
def get_vocabulary(donnees_tokenisees: list[list[str]]) -> set[str]:
    """
    Extrait le vocabulaire à partir d'une liste de données tokenisées.

    Paramètres :
        donnees_tokenisees (list[list[str]]) : Une liste de phrases tokenisées.

    Retourne :
        set[str] : Un ensemble contenant les mots uniques présents dans les données tokenisées.
    """
    vocabulaire = set()
    for sous_liste in donnees_tokenisees:
        for element in sous_liste:
            vocabulaire.add(element)
    return vocabulaire


|<h2>Division des données tokenisées en ensembles d'entraînement et de test</h2> |
| :------------------: |
| La fonction splitting_data prend en entrée une liste de listes de tokens donnees_tokenisees et divise ces données en deux ensembles : un ensemble d'entraînement et un ensemble de test. Le ratio entre ces deux ensembles est déterminé par le paramètre ratio_entrainement. Par défaut, 80% des données sont allouées à l'ensemble d'entraînement et 20% à l'ensemble de test.

La fonction retourne un tuple contenant les deux ensembles, et elle offre également une option d'affichage pour montrer quelques exemples de chaque ensemble si le paramètre affichage est défini sur True..|


In [42]:
def splitting_data(donnees_tokenisees: list[list[str]], ratio_entrainement: float=0.8,
                affichage: bool=False) -> tuple[list[list[str]], list[list[str]]]:
    """
    Divise les données en ensembles d'entraînement et de test en fonction du ratio spécifié.
    
    Paramètres :
        donnees_tokenisees (list[list[str]]) : Les données d'entrée sous forme d'une liste de listes de tokens.
        ratio_entrainement (float) : Le ratio des données à utiliser pour l'entraînement (entre 0 et 1).
        affichage (bool) : Si vrai, des informations sur la division seront affichées.
        
    Retourne :
        Tuple[List[list[str]], List[list[str]]] : Les ensembles d'entraînement et de test sous forme de listes de phrases ou de séquences.
    """
    taille_entrainement = int(len(donnees_tokenisees) * ratio_entrainement)
    donnees_entrainement = donnees_tokenisees[:taille_entrainement]
    donnees_test = donnees_tokenisees[taille_entrainement:]

    if affichage:
        print(f"{len(donnees_tokenisees)} données sont divisées en {len(donnees_entrainement)} ensemble d'entraînement et {len(donnees_test)} ensemble de test")

        print("Premier échantillon d'entraînement :")
        print(donnees_entrainement[0])
            
        print("Premier échantillon de test :")
        print(donnees_test[0])
    
    return donnees_entrainement, donnees_test


|<h2>Vérification de la présence d'un mot dans le vocabulaire</h2> |
| :------------------: |
| La fonction isKnown prend en entrée un mot  et un ensemble de mots vocabulaire. Elle vérifie si le mot (après avoir été converti en minuscules pour éviter les différences de casse) est présent dans le vocabulaire. La fonction retourne True si le mot est présent dans le vocabulaire et False sinon.|


In [43]:
def isKnown(mot: str, vocabulaire: set[str]) -> bool:
    """
    Vérifie si un mot est présent dans le vocabulaire.

    Paramètres :
        mot (str) : Le mot à vérifier.
        vocabulaire (set[str]) : L'ensemble de mots représentant le vocabulaire.

    Retourne :
        bool : True si le mot est dans le vocabulaire, False sinon.
    """
    return mot.lower() in vocabulaire


|<h2>Calcul de la distance d'édition minimale entre deux mots.</h2> |
| :------------------: |
|La fonction calculer_distance_edition détermine la distance d'édition minimale nécessaire pour transformer un mot source en un mot cible. La distance d'édition représente le nombre minimum d'opérations d'insertion, de suppression ou de remplacement nécessaires pour transformer le mot source en le mot cible.Les paramètres cout_insertion, cout_suppression, et cout_remplacement définissent le coût associé à chaque type d'opération. Par défaut, le coût d'insertion est de 1, le coût de suppression est de 1, et le coût de remplacement est de 2. La fonction retourne une matrice D qui stocke les distances d'édition intermédiaires pour chaque sous-séquence des mots source et cible, ainsi que la distance d'édition minimale med entre les deux mots.|


In [44]:
def calculer_distance_edition(source: str, cible: str, cout_insertion: int=1, 
                            cout_suppression: int=1, cout_remplacement: int=2) -> tuple[np.ndarray, int]:
    """
    Calcule la distance d'édition minimale entre deux mots.
    
    Paramètres :
        source (str) : Le mot source.
        cible (str) : Le mot cible.
        cout_insertion (int) : Le coût de l'insertion (par défaut 1).
        cout_suppression (int) : Le coût de la suppression (par défaut 1).
        cout_remplacement (i nt) : Le coût du remplacement (par défaut 2).

    Retourne :
        D : une matrice de dimensions len(source)+1 par len(cible)+1 contenant les distances d'édition minimales
        med : la distance d'édition minimale (med) nécessaire pour convertir la chaîne source en la cible
    """
    m, n = len(source), len(cible)
    D = np.zeros((m+1, n+1), dtype=int)
    
    for ligne in range(1, m+1):
        D[ligne, 0] = D[ligne-1, 0] + cout_suppression
        
    for colonne in range(1, n+1):
        D[0, colonne] = D[0, colonne-1] + cout_insertion
        
    for ligne in range(1, m+1):
        for colonne in range(1, n+1):
            cout_rempl = cout_remplacement
            if source[ligne-1] == cible[colonne-1]:
                cout_rempl = 0
            D[ligne, colonne] = min(D[ligne-1, colonne] + cout_suppression, D[ligne, colonne-1] + cout_insertion, D[ligne-1, colonne-1] + cout_rempl)
          
    med = D[m, n]
    
    return D, med


|<h2>Corrections possibles avec une seule modification.</h2> |
| :------------------: |
| La fonction edits1 génère un ensemble de corrections possibles pour un mot mal orthographié en considérant une seule modification (suppression, insertion, remplacement d'une lettre). Pour chaque type de modification, la fonction parcourt toutes les positions possibles où une modification peut être appliquée, puis génère toutes les corrections potentielles en utilisant l'alphabet. Ensuite, elle filtre ces corrections potentielles pour ne conserver que celles qui sont présentes dans le vocabulaire donné.La fonction retourne un ensemble contenant toutes les corrections possibles pour le mot mal orthographié.|


In [45]:
def edits1(mot: str, vocabulaire: list[str]) -> set[str]:
    """
    Génère un ensemble de corrections possibles pour un mot mal orthographié avec une seule modification.

    Paramètres :
        mot (str) : Le mot mal orthographié.
        vocabulaire (list[str]) : Une liste de mots du vocabulaire.

    Retourne :
        set[str] : Un ensemble de corrections possibles.
    """

    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    decoupes = [(mot[:i], mot[i:]) for i in range(len(mot) + 1)]

    suppressions = [gauche + droite[1:] for gauche, droite in decoupes if droite]
    suppressions = [s for s in suppressions if est_mot(s, vocabulaire)]

    insertions = [gauche + c + droite for gauche, droite in decoupes for c in alphabet]
    insertions = [i for i in insertions if est_mot(i, vocabulaire)]

    remplacements = [gauche + c + droite[1:] for gauche, droite in decoupes if droite for c in alphabet]
    remplacements = [r for r in remplacements if est_mot(r, vocabulaire)]
    
    return set(suppressions + insertions + remplacements)


|<h2>Corrections possibles avec modification >= 1.</h2> |
| :------------: |
| La fonction edits2 génère un ensemble de corrections possibles pour un mot mal orthographié en considérant une ou plusieurs modification (suppression, insertion, remplacement d'une lettre).

In [46]:
def edits2(mot: str, vocabulaire: list[str], n_modifications: int=1, 
                        distance_max: int=2) -> set[str]:
    """
    Génère une liste de corrections possibles pour un mot mal orthographié en fonction du vocabulaire donné et de la distance d'édition maximale.

    Paramètres :
        mot (str) : Le mot mal orthographié.
        vocabulaire (list[str]) : Une liste de mots du vocabulaire.
        n_modifications (int) : Le nombre de modifications autorisées pour générer des corrections (par défaut 1).
        distance_max (int) : La distance d'édition maximale autorisée pour qu'une correction soit considérée (par défaut 2).

    Retourne :
        corrections_possibles (set[str]) : Un ensemble de corrections possibles.
    """
    corrections_possibles = set()

    if n_modifications == 1:
        corrections_possibles = {corr for corr in edits1(mot, vocabulaire) if calculer_distance_edition(mot, corr)[1] <= distance_max}
    else:
        modifications_precedentes = {mot}
        for _ in range(n_modifications):
            modifications_actuelles = set()
            for modification_precedente in modifications_precedentes:
                nouvelles_corrections = {corr for corr in edits1(modification_precedente, vocabulaire) if calculer_distance_edition(mot, corr)[1] <= distance_max}
                modifications_actuelles.update(nouvelles_corrections)
            modifications_precedentes = modifications_actuelles
        corrections_possibles = modifications_precedentes

    return corrections_possibles


|<h2>Comptage de nombre d'occurrences d'un mot dans les données tokenisées</h2> |
| :------------------: |



In [47]:
def compter_mots(phrases_tokenisees: list[list[str]]) -> dict[str, int]:
    """
    Compte le nombre d'apparitions de chaque mot dans les phrases tokenisées.

    Paramètres :
        sentences_tokenises (list[list[str]]) : Liste de listes de chaînes de caractères.

    Retourne :
        word_counts (dict[str, int]) : Dictionnaire qui fait correspondre chaque mot (str) à sa fréquence (int).
    """

    word_counts = {}
    for phrase in phrases_tokenisees:
        for token in phrase:
            if token not in word_counts.keys():
                word_counts[token] = 1
            else:
                word_counts[token] += 1

    return word_counts


|<h2>Traitement des mots hors vocabulaire (OOV)</h2> |
| :------------------: |
 

In [48]:
def obtenir_mots_avec_frequence_superieure_ou_egale(phrases_tokenisees: list[list[str]], 
                                   seuil_frequence: int) -> list[str]:
    """
    Trouve les mots qui apparaissent N fois ou plus.

    Paramètres :
        tokenized_sentences (list[list[str]]) : Liste de listes de phrases.
        seuil_frequence (int) : Nombre minimum d'occurrences pour qu'un mot fasse partie du vocabulaire restreint.

    Retourne :
        closed_vocab (list[str]) : Liste des mots qui apparaissent N fois ou plus.
    """
    closed_vocab = []
    word_counts = compter_mots(phrases_tokenisees)
    for word, cnt in word_counts.items():
        if cnt >= seuil_frequence:
            closed_vocab.append(word)

    return closed_vocab


In [49]:
def remplacer_mots_inconnus_par_unk(phrases_tokenisees: list[list[str]], 
                             vocabulaire: list[str], unknown_token: str="<unk>"
                             ) -> list[list[str]]:
    """
    Replace words not in the given vocabulary with the unknown token.

    Parameters:
        phrases_tokenisees: List of lists of strings
        vocabulaire: List of strings that we will use
        unknown_token: A string representing unknown (out-of-vocabulary) words
    
    Returns:
        replaced_phrases_tokenisees: List of lists of strings, with words not in the vocabulary replaced
    """
    
    vocabulaire = set(vocabulaire)
    replaced_phrases_tokenisees = []
    for phrase in phrases_tokenisees:
        replaced_phrase = []
        for token in phrase:
            if token in vocabulaire:
                replaced_phrase.append(token)
            else:
                replaced_phrase.append(unknown_token)
        replaced_phrases_tokenisees.append(replaced_phrase)
        
    return replaced_phrases_tokenisees


In [50]:
def get_vocabulary_unk(phrases_tokenisees: list[list[str]], seuil_frequence: int):
    """
    Prétraite les données en remplaçant les mots peu fréquents par le jeton inconnu.
    
    Paramètres :
        données (list[list[str]]) : Liste de listes de chaînes de caractères.
        freq_threshold (int) : Les mots dont le compte est inférieur à cette valeur sont considérés comme inconnus.

    Retourne :
        Tuple de
        - données avec les mots peu fréquents remplacés par "<unk>"
        - vocabulaire des mots qui apparaissent au moins n fois dans les données d'entraînement
    """
   
    vocabulaire = obtenir_mots_avec_frequence_superieure_ou_egale(phrases_tokenisees, seuil_frequence)
    données_remplacées = remplacer_mots_inconnus_par_unk(phrases_tokenisees, vocabulaire)
    
    return données_remplacées, vocabulaire


|<h2>Un model de langue n-gramme</h2> |
| :------------------: |

In [51]:
def compter_n_grammes(donnees: list[list[str]], n: int, 
                      jeton_debut: str='<s>', jeton_fin: str= '</s>') -> dict:
    """
    Compte tous les n-grammes dans les données fournies.
    
    Paramètres :
        donnees (list[list[str]]) : Liste de listes de mots.
        n (int) : nombre de mots dans une séquence (par défaut 2).
        jeton_debut (str) : une chaîne de caractères indiquant le début de la phrase (par défaut '<s>').
        jeton_fin (str) : une chaîne de caractères indiquant le fin de la phrase (par défaut '</s>').
    
    Retourne :
        n_grammes (dict) : Un dictionnaire qui mappe un tuple de n mots à sa fréquence.
    """
    n_grammes = {}

    # Parcours des phrases dans les données
    for phrase in donnees:
        # Ajout des jetons de début et de fin à la phrase
        phrase = [jeton_debut] * (n - 1) + phrase + [jeton_fin]
        
        # Génération des n-grammes pour la phrase
        for i in range(len(phrase) - n + 1): 
            n_gramme = tuple(phrase[i:i+n])
            
            # Incrémentation du compteur du n-gramme
            if n_gramme in n_grammes:
                n_grammes[n_gramme] += 1
            else:
                n_grammes[n_gramme] = 1
    
    return n_grammes


|<h2>Un modèle de langue avec Smoothing </h2> |
| :------------------: |

In [52]:
# Fonction pour estimer la probabilité d'un mot donné un contexte
def language_model(word, n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=0.0001):
    """
    Estime la probabilité d'un mot donné un contexte.

    Args:
        word: str - Mot à prédire
        n_gram: tuple - Contexte de mots précédents
        n_gram_counts: dict - Comptes des n-grammes
        n_plus1_gram_counts: dict - Comptes des (n+1)-grammes
        vocabulary_size: int - Taille du vocabulaire
        k: float - Constante de lissage (par défaut 0.01)

    Returns:
        float - Probabilité du mot donné le contexte
    """
    n_gram = tuple(n_gram)
    n_plus1_gram = n_gram + (word,)
    
    # Nombre d'occurrences du n-gramme et du (n+1)-gramme
    count_n_gram = n_gram_counts[n_gram] if n_gram in n_gram_counts else 0
    count_n_plus1_gram = n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts else 0
    
    # Calcul de la probabilité lissée
    numerator = count_n_plus1_gram + k
    denominator = count_n_gram + k * vocabulary_size
    
    probability = numerator / denominator
    return probability

|<h2>Correction d'un texte avec un modèle bigramme</h2> |
| :------------------: |

In [53]:
def correction(texte: str, vocabulaire: set[str], top_n: int=4, n_g: int=2,
               n_edits: int=1, max_distance: int=2) -> tuple[dict, str]:
    """
    Effectue une correction orthographique sur le texte donné en utilisant un modèle de langue n-gramme.

    Paramètres :
        texte (str) : Le texte sur lequel effectuer la correction orthographique.
        vocabulaire (set[str]) : Un ensemble de mots représentant le vocabulaire.
        top_n (int) : Le nombre de suggestions les plus probables à prendre en compte (par défaut 2).
        n_g (int) : L'ordre du modèle de langue n-gramme (par défaut 2).
        k (int) : Constante positive, paramètre de lissage (par défaut 1.0).
        n_edits (int) : Le nombre maximum de modifications autorisées dans une correction suggérée (par défaut 1).
        max_distance (int) : La distance maximale de modification autorisée pour une correction suggérée (par défaut 2).

    Retourne :
        sorted_dict (dict) : Dictionnaire de suggestions.
        texte_corrige (str) : La version corrigée du texte d'entrée.
    """
   
    n_meilleures = []
    suggestions = dict()
    # Traitement du texte
    phrases, phrases_tokenisees = process_data(texte)
    # Comptage des n-grammes
    n_grammes = compter_n_grammes(phrases_tokenisees, n_g)
    n_plus1_grammes = compter_n_grammes(phrases_tokenisees, n_g+1)
    texte_corrige = texte
    
    # Parcours des phrases tokenisées
    for phrase in phrases_tokenisees:
        # Création d'une copie de la phrase avec des marqueurs 
        # de début et de fin pour faciliter la gestion des n-grammes.
        index = None
        phrase_tmp = ['<s>']*(n_g-1) + phrase + ['</s>']
        
        
        for token in phrase:
            probas = dict()
            # Vérification si le token est connu
            if not isKnown(token, vocabulaire):
                index = phrase_tmp.index(token)
                n_gramme_precedent = tuple(phrase_tmp[abs(index-n_g):index])  
                comptes_n_grammes = n_grammes.get(n_gramme_precedent, {}) 
                corrections = edits2(token, vocabulaire, n_edits, max_distance)
                corrections = [c for c in corrections if est_mot(c, vocabulaire)]
                # Calcul des probabilités des corrections
                for corr in corrections:
                    proba = language_model(corr, n_gramme_precedent, n_grammes,
                                           n_plus1_grammes, len(vocabulaire))
                    probas[corr] = proba
                suggestions[token] = probas
        
        # Tri des suggestions par probabilité
        suggestions_triees = {k: dict(sorted(v.items(), key=lambda item: item[1], reverse=True)) for k, v in suggestions.items()}
        sorted_dict = {}
        # Sélection des meilleures t suggestions
        for cle, dict_interne in suggestions_triees.items():
            dict_interne_trie = dict(sorted(dict_interne.items(), key=lambda item: item[1], reverse=True)[:top_n])
            sorted_dict[cle] = dict_interne_trie

        # Remplacement des mots dans le texte corrigé
        for cle in sorted_dict.keys():
            if cle in texte_corrige:
                if sorted_dict[cle]:  # Vérifier si le dictionnaire n'est pas vide
                    premier_mot_interne = next(iter(sorted_dict[cle]))
                    texte_corrige = texte_corrige.replace(cle, premier_mot_interne)

    # Retourner le dictionnaire de suggestions et le texte corrigé
    return sorted_dict, texte_corrige


|<h2>Application</h2> |
| :------------------: |



In [54]:
donnees = load_data('C:\\Users\\hp\\Downloads\\NLP_DL\\big.txt')
phrases, donnees_tokenisees = process_data(donnees)
vocabulaire = get_vocabulary(donnees_tokenisees)

n_gr = 2
n_grams=compter_n_grammes(training_data, n_gr)
n1_grams=compter_n_grammes(training_data, n_gr+1)
vocabulary_size = len(n_grams)

text = "Hallo there, I wqs tryinj to finich this wirk !"

sorted_dict, texte_corrige = correction(text, vocabulaire)

print(f"Le texte original:\n{text}\n")
print(f"Le texte corrigé:\n{texte_corrige}\n")
print(f"Les mots mal orthographiés et leurs corrections:")
for mot in sorted_dict.keys():
    print('-'*46)
    print(f"{mot}:")
    for c, p in sorted_dict[mot].items():
        print(f"{c}:\t{p}")

Le texte original:
Hallo there, I wqs tryinj to finich this wirk !

Le texte corrigé:
Hallo there, I ws trying to finish this work !

Les mots mal orthographiés et leurs corrections:
----------------------------------------------
wqs:
ws:	2.0203652820429932e-05
was:	2.0203652820429932e-05
----------------------------------------------
tryinj:
trying:	2.0203652820429932e-05
----------------------------------------------
finich:
finish:	2.0203652820429932e-05
----------------------------------------------
wirk:
work:	2.0203652820429932e-05
kirk:	2.0203652820429932e-05
dirk:	2.0203652820429932e-05
wick:	2.0203652820429932e-05


|<h2>Evaluation du modèle de langue par la mesure de perplexité</h2> |
| :------------------: |



In [55]:
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=0.0001):
    """
    Calcule la perplexité pour une liste de phrases.

    Args:
        sentence: List[str] - Liste de mots dans la phrase
        n_gram_counts: dict - Comptes des n-grammes
        n_plus1_gram_counts: dict - Comptes des (n+1)-grammes
        vocabulary_size: int - Taille du vocabulaire
        k: float - Constante de lissage (par défaut 0.001)

    Returns:
        float - Perplexité
    """
    n = len(list(n_gram_counts.keys())[0])  # Longueur des n-grammes
    sentence = ["<s>"] * (n-1) + sentence + ["</s>"]  # Ajout de marqueurs de début et de fin
    sentence = tuple(sentence)
    N = len(sentence)
    product_pi = 1.0
    
    for t in range(n, N):  # Parcours de chaque mot dans la phrase
        n_gram = sentence[t-n:t]
        word = sentence[t]
        probability = language_model(word, n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k)
        product_pi *= 1 / probability

    perplexity = product_pi**(1/float(N))
    return perplexity


donnees = load_data('C:\\Users\\hp\\Downloads\\NLP_DL\\big.txt')
phrases, donnees_tokenisees = process_data(donnees)
training_data, test_data = splitting_data(donnees_tokenisees)

# Calcul des comptes des n-grammes et (n+1)-grammes
n_gr = 2
n_grams=compter_n_grammes(test_data, n_gr)
n1_grams=compter_n_grammes(test_data, n_gr+1)
vocabulary_size = len(n_grams)

# Calcul de la perplexité pour le texte de test
perplexity_test = 0
for sentence in test_data:
    perplexity_test = perplexity_test + calculate_perplexity(sentence, n_grams, n1_grams, vocabulary_size)
print("Perplexité pour le texte de test:", perplexity_test/len(test_data))


Perplexité pour le texte de test: 7.0966650225862224
